In [ ]:
# 1. Read the CSV file
# 2. In the content field, do the following:
# - Tokenize and lower-case the text (end result should be an array of tokens where each token is a lexical unit or a punctuation). For example, “He said: ‘Don’t go there!’” => (he, said, :, ‘, don’t, go, there, !, ‘)
# - Remove consecutive spaces and new lines
# - Find and replace URLs with <URL>
# - Find and replace dates with <DATE>
# - Find and replace numbers with <NUM>
# 3. For the metadata fields:
# - Fill all empty fields with a placeholder NULL


In [16]:
import numpy as np
import pandas as pd 
import re
import nltk
from nltk.tokenize import TweetTokenizer, ToktokTokenizer
from collections import Counter 
import itertools
import matplotlib as plt
import csv

In [8]:
def readUrl(url):
    data = pd.read_csv(url)
    return data

def swapUrl(line):
    urlPattern = r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*'
    line = re.sub(urlPattern,' <URL> ', line)
    return line
def swapDates(line):
    re1 = r'[\d]{1,2}(th)? [adfjmnos]\w*[,]?[.]? ([\d]{2,4})?'
    re2 = r'[adfjmnos]\w*[,]?[.]? [\d]{1,2}(th)?[,]? ([\d]{2,4})?'
    re3 = r'[adfjmnos]\w* [\d]{1,2}[,]?[.]?([\d]{2,4})?'
    re4 = r'[\d]{1,2}-[\d]{1,2}-[\d]{2,4}'
    re5 = r'[\d]{1,2}/[\d]{1,2}/[\d]{2,4}'
    re6 = r'[\d]{1,2} [\d]{1,2} [\d]{2,4}'
    re7 = r'[\d].{1,2}.[\d]{1,2}.[\d]{2,4}'
    finReg = [re1, re2, re3, re4, re5, re6, re7]
    for reg in finReg:
        line = re.sub(reg, ' <DATE> ', line)
    return line
def swapNumb(line):
    pattern = r'[\d]+[,]?([\d]+)?'
    line = re.sub(pattern, ' <NUM> ', line)
    return line
dateCount = 0
numbCount = 0
urlCount = 0
def counter(data):
    countDict = {'DATE': 0, 'URL': 0, 'NUM': 0}
    for line in data:
        for key in countDict:
            countDict[key] += line.count('<'+str(key) + '>') 
    return countDict
def wordCount(content, k):
    flatten = list(itertools.chain.from_iterable(content))
    punct = re.compile(r'\W')
    filtered = filter(lambda i: not punct.search(i), flatten)
    counter = Counter(filtered)
    most_freq = counter.most_common(k)
    return most_freq

    

In [17]:
rawData = readUrl('https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv')
token = TweetTokenizer()
tokenizedList = []
for line in rawData['content']:
    line = line.lower()
    pattern = re.compile(r'\s+')
    line = re.sub(pattern, ' ', line)
    line = line.rstrip('\n')
    line = swapUrl(line)
    line = swapDates(line)
    line = swapNumb(line)
    tokenizedList.append(token.tokenize(line))
rawData['content'] = tokenizedList
metaList = []
for line in rawData['meta_keywords']:
    if (line ==  "['']"):
        metaList.append(np.nan)
    else: 
        metaList.append(line)
rawData['meta_keywords'] = metaList
count = counter(rawData['content'])
counted = wordCount(rawData['content'], 1000)
with open('words.csv', 'wb') as csvfile:
    fwriter = csv.writer(csvfile)
    for x in counted:
        fwriter.writerow(x)
   
#rawData.to_csv(r'/Users/oyvinkm/Documents/DATALOGI/DataScience/out.csv', encoding='utf-8', index=False)
#rawData






TypeError: a bytes-like object is required, not 'str'

In [3]:
test = swapUrl('https://www.youtube.com/watch?v=cJymBJ_5iUg')
print(test)
dateTest = []
dateTest.append(swapDates('17/11/1996'))
dateTest.append(swapDates('17 november 1996'))
dateTest.append(swapDates('17-11-1996'))
dateTest.append(swapDates('17 jun 2018'))
dateTest.append(swapDates('nov 17 1996'))
dateTest.append(swapDates('january, 15 '))
dateTest.append(swapDates('17 november '))
dateTest.append(swapDates('jan. 15 2018'))
dateTest.append(swapDates('jan 31.2018'))
dateTest.append(swapNumb('halla 123 hall 732,189'))
t = ['This is a <DATE> and <URL> <URL> comes up <NUM> <NUM> <NUM>', '<DATE> as <DATE> asd <DATE> asd <URL> <NUM>']
dictio = counter(t)
t = [['a', 'c', 'jsd'], ['d', 'a', 'a', 'c']]
print(wordCount(t, 2))
print(dictio)
#for elm in dateTest:
    #print(elm)

 <URL> 
['a', 'c', 'jsd', 'd', 'a', 'a', 'c']
[('a', 3), ('c', 2)]
{'DATE': 4, 'URL': 3, 'NUM': 4}
